<a href="https://colab.research.google.com/github/manamendraJN/fashion-intelligence-platform/blob/feature%2Fbody-measurement-and-sizing/notebooks/body_measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1 - Setup and Environment Configuration

In [1]:
# PURPOSE: Install required packages, import libraries, and configure the environment
# WHY: Sets up all dependencies needed for deep learning model training

import sys
from datetime import datetime, timezone

# Print session information for tracking
print(f"Student: manamendraJN | Date: {datetime.now(timezone. utc).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python:  {sys.version. split()[0]}")

# Install required packages silently (-q flag)
# timm: Pre-trained vision models library
# albumentations: Advanced image augmentation library
! pip install -q timm albumentations

# Core deep learning imports
import torch
import torch.nn as nn  # Neural network layers
import torch.optim as optim  # Optimizers (AdamW)
from torch.utils.data import Dataset, DataLoader  # Data loading utilities
import timm  # Pre-trained models (EfficientNet, ResNet, MobileNet)

# Data processing imports
import numpy as np  # Numerical operations
import pandas as pd  # CSV/dataframe handling
import cv2  # Image loading
from pathlib import Path  # File path operations
import json  # Save/load configuration files

# Image augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Utilities
from tqdm. auto import tqdm  # Progress bars
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Statistical visualizations
from sklearn.metrics import r2_score  # R² metric calculation

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set device (GPU if available, else CPU)
# WHY: GPU is 10-100x faster for deep learning
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Print environment information
print(f"✅ PyTorch {torch.__version__} | Device: {device}")
if torch.cuda.is_available():
    print(f"✅ GPU:  {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB)")
print("🎲 Running without fixed seeds (non-deterministic mode)")
# NOTE: No random seeds = different results each run (but more realistic generalization)

Student: manamendraJN | Date: 2025-12-11 04:53:48
Python:  3.12.12
✅ PyTorch 2.9.0+cu126 | Device: cuda
✅ GPU:  Tesla T4 (15.8 GB)
🎲 Running without fixed seeds (non-deterministic mode)


# Cell 2 - Google Drive Integration and Directory Setup

In [2]:
# PURPOSE: Mount Google Drive and create project directory structure
# WHY:  Persist data/models across Colab sessions and organize outputs

from google.colab import drive

# Mount Google Drive (force_remount=True ensures clean mount)
drive.mount('/content/drive', force_remount=True)

# Create project directory in Colab's temporary storage
# WHY: Faster read/write than Drive, we'll copy results back later
PROJECT_DIR = Path('/content/body_measurement_research')
PROJECT_DIR.mkdir(exist_ok=True)  # exist_ok=True prevents error if already exists

# Change working directory to project folder
import os
os.chdir(PROJECT_DIR)

# Create subdirectories for organized storage
# models:  Saved model weights (. pth files)
# results: JSON files with metrics and statistics
# checkpoints: Intermediate training checkpoints (not used currently)
# visualizations: Generated plots and charts
for d in ['models', 'results', 'checkpoints', 'visualizations']:
    (PROJECT_DIR / d).mkdir(exist_ok=True)

# Point to dataset location in Google Drive
# WHY: Data stays in Drive (permanent), we load from there
DRIVE_DATA_PATH = Path('/content/drive/MyDrive/body_measurement_data/train')
print(f"✅ Setup complete | Data:  {DRIVE_DATA_PATH}")

Mounted at /content/drive
✅ Setup complete | Data:  /content/drive/MyDrive/body_measurement_data/train


# Cell 3 - Load Dataset from CSV Files


In [3]:
# PURPOSE: Load and merge three CSV files containing photos, measurements, and metadata
# WHY: We need to combine subject IDs with their photos and body measurements

# Load three separate CSV files
# 1. subject_to_photo_map. csv: Maps subject_id → photo_id
# 2. measurements. csv: Contains 14 body measurements per subject_id
# 3. hwg_metadata.csv: Contains height, weight, gender per subject_id
photos_df = pd.read_csv(DRIVE_DATA_PATH / 'subject_to_photo_map.csv')
measurements_df = pd.read_csv(DRIVE_DATA_PATH / 'measurements.csv')
metadata_df = pd.read_csv(DRIVE_DATA_PATH / 'hwg_metadata.csv')

# Merge dataframes on 'subject_id' column
# WHY: We need photo_id, measurements, and metadata in one table
# how='inner':  Only keep subjects that exist in ALL three files
data = photos_df.merge(measurements_df, on='subject_id', how='inner')
data = data.merge(metadata_df, on='subject_id', how='inner')

# Extract measurement column names (exclude non-measurement columns)
# WHY: We need to know which columns are our prediction targets
exclude_cols = ['subject_id', 'Unnamed: 0']  # Remove ID columns
MEASUREMENT_COLUMNS = [col for col in measurements_df.columns if col not in exclude_cols]
# Result: 14 measurements (chest, waist, hip, shoulder, arm, leg, etc.)

print(f"✅ Loaded {len(data)} samples | {len(MEASUREMENT_COLUMNS)} measurements")

✅ Loaded 6134 samples | 14 measurements
